# Time Calibration (Delay Correction)

Please set the correct logfile and calibration file locations under __part 2__

This script will create a new folder named __*name*_data-corrected__ in the folder where the Impulse logfile is located.



## Part 1. Importing the necessary libraries

In [1]:
%matplotlib qt
import matplotlib.pyplot as plt

import csv
from datetime import date, datetime, time, timedelta
from time import time
import pandas as pd
import numpy as np
import math
import os
from pathlib import Path
from ast import literal_eval
import pandas as pd
import random
import scipy.interpolate

## Part 2. Load Logfile and calibration file
When you run this cell a popup opens where you can load your Impulse logfile and a time delay calibration file.
When you place the calibration file inside the same folder as this script, the script will automatically pick that calibration file.

In [2]:
import sys
import PySimpleGUI as sg

files = os.listdir('.')
calibrationFilename = "timeDelayCalibration.csv"
calibrationFile = ""
for file in files:
    if file.endswith(calibrationFilename):
        calibrationFile=file

layout = [[sg.T("Please load the files below:")], [sg.Text("Impulse logfile: "), sg.Input(), sg.FileBrowse(key="-DATA-")], [sg.Text("Calibration file: "), sg.Input(calibrationFile), sg.FileBrowse(key="-CAL-")],[sg.Button("Process")]]

###Building Window
window = sg.Window('Load files', layout, size=(600,150))
    
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event=="Exit":
        sys.exit("No files loaded.")
    elif event == "Process":
        window.close()
        break

if values['-DATA-']=="":
    sys.exit("No Impulse file loaded.")
else:
    impulseLogfilePath = Path(values['-DATA-'])
    
if values['-CAL-'] != "":
    timeDelayCalibrationPath = Path(values['-CAL-'])
else:
    timeDelayCalibrationPath = calibrationFile

#############################################
MSLogfilePath = ""

beforeTemParameters = ["TimeStamp", "Experiment time", "MFC1 Measured", "MFC1 Setpoint","MFC2 Measured", "MFC2 Setpoint","MFC3 Measured", "MFC3 Setpoint", "MixValve", "% Gas1 Measured", "% Gas2 Measured", "% Gas3 Measured", "% Gas1 Setpoint", "% Gas2 Setpoint", "PumpRotation",  "ActiveProgram"]
inTemParameters = ["TimeStamp", "Experiment time", "Fnr", "Fnr Setpoint", "Temperature Setpoint","Temperature Measured", "Pin Measured", "Pin Setpoint", "Pout Measured", "Pout Setpoint", "Pnr (Calculated from Pin Pout)", "Pnr Setpoint","Measured power", "Pvac", "Relative power reference", "Relative power"]
afterTemParameters = ["TimeStamp", "Experiment time", "Channel#1", "Channel#2", "Channel#3", "Channel#4", "Channel#5", "Channel#6", "Channel#7", "Channel#8", "Channel#9", "Channel#10"]

## Part 3. Time Delay calculation functions

It is possible to change the flow/time delay curve parameters here.

In [3]:
if timeDelayCalibrationPath!="":
    print("Loaded curve parameters used.")
    curveParameters = pd.read_csv(timeDelayCalibrationPath)
    beforeCurveParameters = curveParameters['PtI'][0]
    afterCurveParameters = curveParameters['ItP'][0]
    while "  " in beforeCurveParameters: beforeCurveParameters = beforeCurveParameters.replace("  ", " ")
    while "  " in afterCurveParameters: afterCurveParameters = afterCurveParameters.replace("  ", " ")
    beforeCurveParameters = [float(i) for i in beforeCurveParameters.strip("[]").strip().split(" ")]
    afterCurveParameters = [float(i) for i in afterCurveParameters.strip("[]").strip().split(" ")]
    
else:
    print("Default curve parameters used.")
    beforeCurveParameters = [41.0706, 1184.39, -2288.09, 2533.44, 7.73936, 6.38112]
    afterCurveParameters = [25.2776, 690.819, -572.967, 174.849, 10.0311, -20.8556]

def calculateBeforeOffset(P, F): # Calculates the time delay between before-TEM and inside-TEM
    a,b,c,d,e,f = beforeCurveParameters
    return f+ (P*e)* (1/(a+b*F+c*F**2+d*F**3))

def calculateAfterOffset(P, F): # Calculates the time delay between inside-TEM and after-TEM
    a,b,c,d,e,f = afterCurveParameters
    return f+ (P*e)* (1/(a+b*F+c*F**2+d*F**3))

Loaded curve parameters used.


## Part 4. Load logfiles

Nothing needs to be done here, just run the following code.

In [4]:
#Load the Impulse logfile into a pandas dataframe
allData = pd.read_csv(impulseLogfilePath, infer_datetime_format=True)
allData['TimeStamp']= pd.to_datetime(allData['TimeStamp']).dt.time
    
#Separate parameters into before in and after TEM data
beforeTemData = allData.filter(items=beforeTemParameters)
inTemData = allData.filter(items=inTemParameters)
afterTemData = None
if 'Channel#1' in allData: afterTemData = allData.filter(items=afterTemParameters)
    
#If there is a separate MS logfile, load that one (which is a messy business)
if MSLogfilePath!="": 
    headerItems = ["Experiment time"]
    channelSection = 0
    lineCounter = 0
    with open(MSLogfilePath, 'r') as csvfile:
        for line in csvfile:
            line = line.strip()
            if line[:10]=="Start time":
                timeString = line[26:-3]
                timeString = timeString + ".000"
                MSstartTime=datetime.strptime(timeString,"%H:%M:%S.%f")
                if line[-2:]=="PM":
                    MSstartTime = MSstartTime + timedelta(hours=12)
            if line[:7]=="Time(s)":
                channelSection = 0
                headerLength = lineCounter
            if channelSection == 1:
                while '  ' in line:
                    line = line.replace('  ',',')
                while ',,' in line:
                    line = line.replace(',,',',')            
                line = line.split(',')
                if len(line) > 1:    
                    headerItems.append(line[2])
            if line[:7]=="Channel":
                channelSection = 1
            lineCounter=lineCounter+1
    afterTemData = pd.read_csv(MSLogfilePath, header=0, index_col=False, names=headerItems, skiprows=headerLength)
    
    #Calculate the true Impulse starttime (first timestamp - first experiment time seconds)
    impulseExpTimStartMil, impulseExpTimStartSec = math.modf(allData['Experiment time'].iloc[0])
    impulseExpTimStartMil = round(impulseExpTimStartMil,3)*1000
    impulseFirstTimeStamp = datetime.combine(date.today(),allData['TimeStamp'].iloc[0])
    realImpulseStartTime = impulseFirstTimeStamp - timedelta(seconds=impulseExpTimStartSec, milliseconds=impulseExpTimStartMil)
    
    # Calculate the number of seconds offset between the experiment time of the MS and the experiment time of Impulse
    if MSstartTime > realImpulseStartTime:
        experimentTimeOffset = (MSstartTime - realImpulseStartTime).seconds + ((MSstartTime - realImpulseStartTime).microseconds/1000000)
    else:
        experimentTimeOffset = -((realImpulseStartTime-MSstartTime).seconds + ((realImpulseStartTime-MSstartTime).microseconds/1000000))
    
    # Calculate the MS TimeStamps based on MSstartTime and the experiment time, and adjust the experiment time with the offset from Impulse
    afterTemData.insert(0,'StartTime',MSstartTime)
    afterTemData.insert(0,'Experimentsec','')
    afterTemData.insert(0,'TimeStamp','')
    afterTemData['Experimentsec']=pd.to_timedelta(afterTemData['Experiment time'] ,'s')
    afterTemData['TimeStamp']=(afterTemData['StartTime']+afterTemData['Experimentsec']).dt.time
    del afterTemData['StartTime']
    del afterTemData['Experimentsec']
    afterTemData['Experiment time']=afterTemData['Experiment time']+experimentTimeOffset

## Part 5. Remove time offset

Nothing needs to be done here, just run the following code.

In [5]:
#Calculate rolling average for Flow to prevent sudden changes from timewarping the data
RAwindow=5
inTemData['Fnr RA'] = inTemData['Fnr'].rolling(window=RAwindow,center=True).mean()
inTemData['Fnr RA'].fillna(inTemData['Fnr'], inplace=True) #Fill the missing Fnr RA values at the head and tail with the original values

#Correct beforeTemData
beforeTemDataCorrected = beforeTemData.copy()
beforeTemDataCorrected['Fnr RA']=inTemData['Fnr RA']
beforeTemDataCorrected['Pressure']=inTemData['Pnr (Calculated from Pin Pout)']
beforeTemDataCorrected['Time correction (seconds)']=np.vectorize(calculateBeforeOffset)(beforeTemDataCorrected['Pressure'],beforeTemDataCorrected['Fnr RA'])
beforeTemDataCorrected['Time correction timedelta']= pd.to_timedelta(beforeTemDataCorrected['Time correction (seconds)'] ,'s')
beforeTemDataCorrected['TimeStamp']= (pd.to_datetime(beforeTemDataCorrected['TimeStamp'].astype(str))+beforeTemDataCorrected['Time correction timedelta']).dt.time
beforeTemDataCorrected['Experiment time']+=beforeTemDataCorrected['Time correction (seconds)']
del beforeTemDataCorrected['Time correction timedelta']
del beforeTemDataCorrected['Fnr RA']

#Correct afterTemData
if afterTemData is not None:
    afterTemDataCorrected = afterTemData.copy()
    if MSLogfilePath=="": #If the MS data was included in the Impulse logfile (same timestamps)
        afterTemDataCorrected['Fnr RA']=inTemData['Fnr RA']
        afterTemDataCorrected['Pressure']=inTemData['Pnr (Calculated from Pin Pout)']

    if MSLogfilePath!="": #Different logfile for MS, so Fnr RA has to be interpolated
        # Interpolate Fnr RA to calculate offsets for MS data
        FnrRAInterp = scipy.interpolate.interp1d(inTemData['Experiment time'],inTemData['Fnr RA'])
        PnrInterp = scipy.interpolate.interp1d(inTemData['Experiment time'],inTemData['Pnr (Calculated from Pin Pout)'])
        
        #Crop MS logfile so that Experiment time values fall within interpolated range
        minTime = inTemData['Experiment time'].iloc[0]
        maxTime = inTemData['Experiment time'].iloc[-1]
        print(f"mintime {minTime}, maxtime {maxTime}")
        afterTemDataCorrected=afterTemDataCorrected[afterTemDataCorrected['Experiment time']>minTime]
        afterTemDataCorrected=afterTemDataCorrected[afterTemDataCorrected['Experiment time']<maxTime]

        #Find the Fnr RA values for the MS timestamps
        afterTemDataCorrected['Fnr RA']=np.vectorize(FnrRAInterp)(afterTemDataCorrected['Experiment time'])
        afterTemDataCorrected['Pressure']=np.vectorize(PnrInterp)(afterTemDataCorrected['Experiment time'])
        

    afterTemDataCorrected['Time correction (seconds)']=np.vectorize(calculateAfterOffset)(afterTemDataCorrected['Pressure'],afterTemDataCorrected['Fnr RA'])
    afterTemDataCorrected['Time correction timedelta']= pd.to_timedelta(afterTemDataCorrected['Time correction (seconds)'] ,'s')
    afterTemDataCorrected['TimeStamp']= (pd.to_datetime(afterTemDataCorrected['TimeStamp'].astype(str))-afterTemDataCorrected['Time correction timedelta']).dt.time
    afterTemDataCorrected['Experiment time']-=afterTemDataCorrected['Time correction (seconds)']
    del afterTemDataCorrected['Time correction timedelta']
    del afterTemDataCorrected['Fnr RA']

#Function to format the timestamp the way we like it (with only 3 digits for ms)
def timeStampFormatter(dt):
    return "%s:%.3f%s" % (
        dt.strftime('%H:%M'),
        float("%.3f" % (dt.second + dt.microsecond / 1e6)),
        dt.strftime('%z')
    )

#Run the timeStampFormatter for the after and before datasets
if afterTemData is not None: afterTemDataCorrected['TimeStamp']= np.vectorize(timeStampFormatter)(afterTemDataCorrected['TimeStamp'])
beforeTemDataCorrected['TimeStamp']= np.vectorize(timeStampFormatter)(beforeTemDataCorrected['TimeStamp'])

## Part 6. Plot corrected data

The corrected data is visualized here to check if the correction was done correctly.

You can choose which parameters from each of the datasets (before, in, after) will be plotted by changing the 3 strings at the top of the following code. Make sure that the string content exactly matches the header name.

In [34]:
preTEMparameter ='MFC1 Measured'
inTEMparameter = 'Measured power'
postTEMparameter = 'Channel#1'


from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
import matplotlib.pyplot as plt

host = host_subplot(111, axes_class=AA.Axes)
plt.subplots_adjust(right=0.75)

par1 = host.twinx()
par2 = host.twinx()

offset = 60
new_fixed_axis = par2.get_grid_helper().new_fixed_axis
par2.axis["right"] = new_fixed_axis(loc="right",
                                    axes=par2,
                                    offset=(offset, 0))

par1.axis["right"].toggle(all=True)
par2.axis["right"].toggle(all=True)

host.set_xlabel("Experiment Time")
host.set_ylabel(inTEMparameter)
par1.set_ylabel(postTEMparameter)
par2.set_ylabel(preTEMparameter)

p1, = host.plot(inTemData['Experiment time'],inTemData[inTEMparameter], label=inTEMparameter)

p2, = par1.plot(afterTemDataCorrected['Experiment time'],afterTemDataCorrected[postTEMparameter],label=f"{postTEMparameter} corrected")
p2o, = par1.plot(afterTemData['Experiment time'],afterTemData[postTEMparameter], label=f"{postTEMparameter} original", color=p2.get_color(), linestyle="dotted", alpha=0.4)

p3, = par2.plot(beforeTemDataCorrected['Experiment time'],beforeTemDataCorrected[preTEMparameter], label=f"{preTEMparameter} corrected")
p3o, = par2.plot(beforeTemData['Experiment time'],beforeTemData[preTEMparameter], label=f"{preTEMparameter} original", color=p3.get_color(), linestyle="dotted", alpha=0.4)

host.grid(b=True, which='major', color='#666666', linestyle='-')
host.minorticks_on()
host.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)


host.axis["left"].label.set_color(p1.get_color())
par1.axis["right"].label.set_color(p2.get_color())
par2.axis["right"].label.set_color(p3.get_color())

host.legend()
host.autoscale()
host.set_xlim(inTemData['Experiment time'].min(),inTemData['Experiment time'].max())

host.toolbar_visible = False
host.header_visible = False
host.resizable = True

host.title.set_text('Calibration result')

plt.draw()
plt.show()

## Part 7. Save new CSV files

To save the corrected CSV files, run the code below.

In [7]:
# Paths and filenames
experimentName = os.path.splitext(impulseLogfilePath.name)[0]
correctedDataFolder = os.path.dirname(impulseLogfilePath)#+"/"+experimentName+"_corrected-data"
Path(correctedDataFolder).mkdir(parents=True, exist_ok=True) # Create corrected-data folder

#Create the CSV files
inTemData.to_csv((experimentName+'_corrected-inside.csv'), index=False)
beforeTemData.to_csv((experimentName+'_corrected-before.csv'), index=False)            
if afterTemData is not None: afterTemData.to_csv((experimentName+'_corrected-after.csv'), index=False)
   
beforeTemDataCorrected = beforeTemDataCorrected.sort_values(by = 'Experiment time')
inTemData = inTemData.sort_values(by = 'Experiment time')
syncData = pd.merge_asof(inTemData, beforeTemDataCorrected, on = 'Experiment time')
if afterTemData is not None:
    afterTemDataCorrected = afterTemDataCorrected.sort_values(by = 'Experiment time')
    syncData = pd.merge_asof(syncData, afterTemDataCorrected, on = 'Experiment time')

syncData.to_csv((experimentName+'_corrected-synchronized.csv'), index=False)
